# Installation

## Install dependecies

In [ ]:
update = False

from os import path


%cd /kaggle/working
!git clone https://github.com/comfyanonymous/ComfyUI.git
%cd ComfyUI
if update:
    get_ipython().system('git pull')
!pip install -r requirements.txt

checkpoints =  '/kaggle/working/ComfyUI/models/checkpoints'
link_path = checkpoints + '/temp-models'
temp_models = '/kaggle/temp/temp-models'

!mkdir /kaggle/temp
!mkdir $temp_models

if not path.exists(link_path):
    get_ipython().system(f'ln -s {temp_models} {checkpoints}')

!mamba install openssh -y
# !micromamba install openssh -y


# Install the node manager
update_manager = False
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/ltdrdata/ComfyUI-Manager.git
%cd ComfyUI-Manager
if update_manager:
    get_ipython().system('git pull')
!pip install -r requirements.txt

In [ ]:
!pip install ultralytics==8.3.45
!pip install comfy-cli

In [ ]:
!echo "yes" | comfy node update all
!echo "yes" | comfy node install-deps \
--workflow="/kaggle/input/virtual-tryon-workflow/Virtual_TryOn.json"
!echo "yes" | comfy node update all
!echo "yes" | comfy set-default "/kaggle/working/ComfyUI"

In [1]:
!rm -rf /kaggle/working/ComfyUI/custom_nodes/ComfyUI-Yolo-Cropper
%cd /kaggle/working/ComfyUI/custom_nodes
!git clone https://github.com/HongQuan1812/ComfyUI-Yolo-Cropper.git

/kaggle/working/ComfyUI/custom_nodes
Cloning into 'ComfyUI-Yolo-Cropper'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 38 (delta 15), reused 12 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (38/38), 2.24 MiB | 38.86 MiB/s, done.
Resolving deltas: 100% (15/15), done.


---
# Model Management

## Install a model

Copy the model URL to the model_url field. Make sure the model can be accessed publicly, without being signed into a website.

In [ ]:
def install_civitai_model(api_key, model_version_id, 
                          place=None, model_name=None, 
                          type=None, format=None, size=None, fp=None):
    
    url_base = f'https://civitai.com/api/download/models/{model_version_id}'
    model_url = url_base

    if any([type, format, size, fp]):
        model_url += '?'
    
        # Build the query parameters
        conditions = []
        if type is not None:
            conditions.append(f"type={type}")
        if format is not None:
            conditions.append(f"format={format}")
        if size is not None:
            conditions.append(f"size={size}")
        if fp is not None:
            conditions.append(f"fp={fp}")
    
        # Join the parameters with '&'
        query_string = "&".join(conditions)

        model_url += query_string
        model_url += f"&token={api_key}"
        
    else:
        model_url += f"?token={api_key}"
        

    if model_name is not None:
        command = f'wget -O "{model_name}" "{model_url}"'
    else:
        command = f'wget "{model_url}"'
    
    # print(model_url)
    try: 
        %cd $place
        get_ipython().system(command)
        print("-----------------------------------")
        print("| Downloading process is complete |")
        print("-----------------------------------")
    except Exception as err:
        print(f"An error occurred: {err}")

api_key = "398343858e0078ffee8f0967b3531105"

In [ ]:
import os

def ensure_directory_exists(directory_path):
    """
    Ensures that the specified directory exists. 
    If it doesn't exist, it creates the directory.

    Args:
        directory_path (str): The path of the directory to check or create.

    Returns:
        str: A message indicating whether the directory was created or already exists.
    """
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)  # Create the folder
        return f"Folder '{directory_path}' created."
    else:
        return f"Folder '{directory_path}' already exists."


In [ ]:
model_folder_path = "/kaggle/working/ComfyUI/models/ipadapter"
ensure_directory_exists(model_folder_path)
model_name = "ip-adapter-plus_sdxl_vit-h.safetensors"

model_path = os.path.join(model_folder_path, model_name)
model_url = "https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter-plus_sdxl_vit-h.safetensors"

!wget -O $model_path $model_url

In [ ]:
model_folder_path = "/kaggle/working/ComfyUI/models/clip_vision"
ensure_directory_exists(model_folder_path)
model_name = "CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors"

model_path = os.path.join(model_folder_path, model_name)
model_url = "https://huggingface.co/laion/CLIP-ViT-H-14-laion2B-s32B-b79K/resolve/main/model.safetensors"

!wget -O $model_path $model_url

In [ ]:
model_folder_path = "/kaggle/working/ComfyUI/models/mediapipe"
ensure_directory_exists(model_folder_path)
model_name = "selfie_multiclass_256x256.tflite"

model_path = os.path.join(model_folder_path, model_name)
model_url = "https://storage.googleapis.com/mediapipe-models/image_segmenter/selfie_multiclass_256x256/float32/latest/selfie_multiclass_256x256.tflite"

!wget -O $model_path $model_url

In [ ]:
# Install a Eyeful checkpoints in permanent storage

place = "/kaggle/working/ComfyUI/models/yolov8"
message = ensure_directory_exists(place)
print(message)

model_version_id = 582143
model_name="eyefulRobustEyeDetection_v2Paired.zip"

install_civitai_model(api_key=api_key, model_version_id=model_version_id, 
                       place=place, model_name=model_name)

!unzip eyefulRobustEyeDetection_v2Paired.zip

In [ ]:
# Install a Foocus Inpaint model in permanent storage

model_folder_path = "/kaggle/working/ComfyUI/models/inpaint"
ensure_directory_exists(model_folder_path)

model_name = "fooocus_inpaint_head.pth"
model_path = os.path.join(model_folder_path, model_name)
model_url = "https://huggingface.co/lllyasviel/fooocus_inpaint/resolve/main/fooocus_inpaint_head.pth"
!wget -O $model_path $model_url

model_name = "inpaint_v26.fooocus.patch"
model_path = os.path.join(model_folder_path, model_name)
model_url = "https://huggingface.co/lllyasviel/fooocus_inpaint/resolve/main/inpaint_v26.fooocus.patch"
!wget -O $model_path $model_url

In [ ]:
# # Install a controlnet model in permanent storage

# model_folder_path = "/kaggle/working/ComfyUI/models/controlnet"
# ensure_directory_exists(model_folder_path)
# model_folder_path = "/kaggle/working/ComfyUI/models/controlnet/SDXL"
# ensure_directory_exists(model_folder_path)
# model_folder_path = "/kaggle/working/ComfyUI/models/controlnet/SDXL/controlnet-union-sdxl-1.0"
# ensure_directory_exists(model_folder_path)

# model_name = "diffusion_pytorch_model_promax.safetensors"
# model_path = os.path.join(model_folder_path, model_name)
# model_url = "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors"
# !wget -O $model_path $model_url

In [ ]:
# Install a model in permanent storage
# Make sure Persistence is set to "Files only" or "Variables and Files"

checkpoints =  '/kaggle/working/ComfyUI/models/checkpoints'

# model_name = 'Juggernaut XL inpainting.safetensors'
# model_version_id = 456538
# install_civitai_model(api_key=api_key, model_version_id=model_version_id, 
#                       place=checkpoints, model_name=model_name)

# model_name = 'Juggernaut XL.safetensors'
# model_version_id = 782002
# install_civitai_model(api_key=api_key, model_version_id=model_version_id, 
#                       place=checkpoints, model_name=model_name)

model_name = 'Realism Engine SDXL.safetensors'
model_version_id = 293240
install_civitai_model(api_key=api_key, model_version_id=model_version_id, 
                      place=checkpoints, model_name=model_name)


In [ ]:
# %rm "/kaggle/working/ComfyUI/models/checkpoints/Juggernaut XL.safetensors"

In [ ]:
# # Install a LoRA in permanent storage
# place = 'kaggle/working/ComfyUI/models/loras'
# model_name = 'DreamArt.safetensors'
# model_version_id = 137124
# type="Model"
# format="SafeTensor"

# install_civitai_model(api_key=api_key, model_version_id=model_version_id, 
#                       place=place, model_name=model_name,
#                      type=type, format=format)

In [ ]:
##Install a model in temporary storage
# model_url = 'https://civitai.com/api/download/models/160191?type=Model&format=SafeTensor&size=full&fp=fp16'
# model_name = 'YamersRealism.safetensors'
# model_url = 'https://civitai.com/api/download/models/456751'
# model_name = 'HelloWorld-XL.safetensors' 

# %cd $temp_models
# get_ipython().system(f'wget -O "{model_name}" "{model_url}"')

--- 
# WebUI

## Start the WebUI with Pinggy
* Wait for the GUI to start.  
* Click the link that ends with .pinggy.link 😁
* If generation is still running after the link expires in an hour, wait for the generation to complete and restart the WebUI code block to get a new link

In [3]:
# Starting the Web UI with pinggy

from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    cmd = f"python /kaggle/working/ComfyUI/main.py --force-fp16 & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8188 a.pinggy.io > log.txt"
    get_ipython().system(cmd)
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()

waiting for output
[START] Security scan
waiting for output
Allocated port 3 for remote forward to localhost:8188
[DONE] Security scan
## ComfyUI-Manager: installing dependencies done.
** ComfyUI startup time: 2024-12-25 08:36:39.762824
** Platform: Linux
** Python version: 3.10.13 | packaged by conda-forge | (main, Dec 23 2023, 15:36:39) [GCC 12.3.0]
** Python executable: /opt/conda/bin/python
** ComfyUI Path: /kaggle/working/ComfyUI
** Log path: /kaggle/working/ComfyUI/custom_nodes/comfyui.log
😁 😁 😁
URL: http://rnqnm-35-244-103-46.a.free.pinggy.link
😁 😁 😁
7=)0]8;;\                                                                                                                                                                                                                                                                                                                                                                                                                                      

KeyboardInterrupt: 

Connection to a.pinggy.io closed by remote host.
Connection to a.pinggy.io closed.


# Workflow to Code

## Install

In [ ]:
# %cd /kaggle/working/ComfyUI/custom_nodes
# !git clone https://github.com/atmaranto/ComfyUI-SaveAsScript.git
# %cd ComfyUI-SaveAsScript
# !pip install -r requirements.txt

In [ ]:
# %cd /kaggle/working/ComfyUI/custom_nodes
# !git clone https://github.com/pydn/ComfyUI-to-Python-Extension.git
# %cd ComfyUI-to-Python-Extension
# !pip install -r requirements.txt

In [ ]:
# %cd /kaggle/working/ComfyUI/custom_nodes/ComfyUI-to-Python-Extension
# !python comfyui_to_python.py -h

In [ ]:
# %cd /kaggle/working/ComfyUI/custom_nodes/ComfyUI-to-Python-Extension
# !python comfyui_to_python.py \
# -f "/kaggle/input/virtual-tryon-workflow/Virtual_TryOn_api(1).json" \
# -o "/kaggle/working/ComfyUI/workflow.py"

In [ ]:
# %cd /kaggle/working/ComfyUI/custom_nodes/ComfyUI-SaveAsScript
# !python comfyui_to_python.py \
# "/kaggle/input/virtual-tryon-workflow/Virtual_TryOn_api(1).json" \
# --output "/kaggle/working/ComfyUI/workflow.py" -y

In [ ]:
# !python "/kaggle/working/ComfyUI/workflow.py" -h

## Run

In [ ]:
# !python /kaggle/working/ComfyUI/workflow.py \
# "/kaggle/working/Images/Taylor_Swift_at_the_Golden_Globes_2024_(Enhanced,_cropped)_1.jpg" \
# "/kaggle/working/Images/30577753.jpeg.png" --queue-size 1 --



---
# File Browser

##  Install FileBrowser

In [ ]:
%cd /kaggle
!wget https://github.com/filebrowser/filebrowser/releases/download/v2.27.0/linux-amd64-filebrowser.tar.gz
!tar xvfz linux-amd64-filebrowser.tar.gz
!chmod a+x /kaggle/filebrowser
!/kaggle/filebrowser config init 
!/kaggle/filebrowser config set --auth.method=noauth > /dev/null
!/kaggle/filebrowser config set --branding.theme=dark > /dev/null
!/kaggle/filebrowser users add admin admin 
!/kaggle/filebrowser config export "/kaggle/config.json"

## Run FileBrowser

In [ ]:
%cd /kaggle
        
from multiprocessing import Process
import sys
import time

!touch log.txt
open('log.txt', 'w').close()

def run_app():
    !/kaggle/filebrowser -c "/kaggle/config.json" & ssh -o StrictHostKeyChecking=no -p 80 -R0:localhost:8080 a.pinggy.io > log.txt > log.txt
    
def print_url():
    print("waiting for output")
    time.sleep(2)
    sys.stdout.flush()
    
    found = False
    with open('log.txt', 'r') as file:
        end_word = '.pinggy.link'
        for line in file:
            start_index = line.find("http:")
            if start_index != -1:
                end_index = line.find(end_word, start_index)
                if end_index != -1:
                    print("😁 😁 😁")
                    print("URL: " + line[start_index:end_index + len(end_word)])
                    print("😁 😁 😁")
                    found = True
    if not found:
        print_url()
    else:
        with open('log.txt', 'r') as file:
            for line in file:
                print(line)
    
p_app = Process(target=run_app)
p_url = Process(target=print_url)
p_app.start()
p_url.start()
p_app.join()
p_url.join()